In [1]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [2]:
import hashlib

In [3]:
! pip3 install cryptography

In [4]:
import base64
import logging

from cryptography.exceptions import UnsupportedAlgorithm
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [5]:
! pip3 install pyDES

In [6]:
import pyDes

# Symmetric Encryption 

use of the DES Algorithm

https://en.wikipedia.org/wiki/Data_Encryption_Standard

In [7]:
def encript(msg, key):
    k = pyDes.des(key, pyDes.CBC, "\0\0\0\0\0\0\0\0", pad=None, padmode=pyDes.PAD_PKCS5)
    return k.encrypt(msg)

In [8]:
def decript(msg, key):
    k = pyDes.des(key, pyDes.CBC, "\0\0\0\0\0\0\0\0", pad=None, padmode=pyDes.PAD_PKCS5)
    return k.decrypt(msg)

In [9]:
# Key must be 8 Bytes
key = "12345678"
msg = "Super secret message!"
encripted_msg = encript(msg, key)
print(encripted_msg)

b"\xda\xd2\xb3\xba\xe5V\xd4\x00\x89\x1fO\xd2\xe5\x13i\xf0\xf1\x98*\xec{J'\x1d"


In [10]:
decripted_msg = decript(encripted_msg, key)
print(decripted_msg)

b'Super secret message!'


# Asymmetric Cryptography

use of the RSA Algorithm (public-key cryptography)

https://en.wikipedia.org/wiki/Public-key_cryptography 

In [11]:
def generate_keys():
    # Generate new key pair
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

In [12]:
private_key, public_key = generate_keys()

In [13]:
pem = public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo )
pem

b'-----BEGIN PUBLIC KEY-----\nMIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAv0xpbiTc2elluoKhnCgf\n9YdCC9DQFnEWq8398/6Z8RTSiXrac56RSiSLjB8kXXAaJjiiLwqL871nJ5aIy8Ni\nrkGY8Zp7TXQrDwqnpF9M1uZ97y+oeMwcf6zclMAMUuySER3Sre6WyYxipuol9UE/\nHNVTCrMVGbIlx18X65GRTcFZHRR9h+Xktd5NlX9IV6dSPyx2DC78r1GQnQumxm22\nBSS+FMGG0A4aWDxruwGmZStvU/EWS0+5Zli1CNBWAAzB7/vUHMNLYwyrQiVoJLsh\nQZX9ObiLwDSfnKI3wPsYWSb6YsWWkuWa1IapcnqBr8Y5/km0ER8o4bAfZCpya7xX\nOAs9i3OR0TMfuSAuZqIjqnddcxNUFKb/1zKzWXv/btNaisZt8NSr9UzXFJSGb6FV\nf8qNx3wqfHPhdjeLjIyGtDb+Bnv1wtWf9asGnmHzY5puJvsjKNjcJ6EJE/z9jnIb\naDlBomd2/Vg7T+Bg03sghSFnaAHNfblywHLY0ic1bLRsc8gMWvQn5wvvmS0ifuNT\nTShAhe8H1iiChW6zGxsTZTSFn6x1cX4XM4b2NFQwsD9Kt0L+I2oj+QsNA8EoExEA\nXHwNtc9NkWqS5d9ftqbIMhizR8+hZpzKxPX38nKDCsMINNlLlkkONGG/P2FN1C33\nKjQr0n+pLJ7H6xn7zyq7Bf0CAwEAAQ==\n-----END PUBLIC KEY-----\n'

In [14]:
def encrypt_asym(plain_text , publickey):
    # Encription with the use of the RSA Library
    cipher_text_bytes = public_key.encrypt(
        plaintext=plain_text.encode('utf-8'),
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )
    # Conversion of raw bytes to BASE64 representation
    cipher_text = base64.urlsafe_b64encode(cipher_text_bytes)
    return cipher_text

def decrypt_asym(cipher_text, private_key):
    # Decription with the use of the RSA Library
    decrypted_cipher_text_bytes = private_key.decrypt(
        ciphertext=base64.urlsafe_b64decode(cipher_text),
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )
    # Conversion to UTF-8 Text
    decrypted_cipher_text = decrypted_cipher_text_bytes.decode('utf-8')
    return decrypted_cipher_text

In [15]:
msg = "This message was encripted with asymmetric cryptography"
encrypted_msg = encrypt_asym(msg , public_key)
encrypted_msg

b'm2G-UDdECN-85EOxUD7prHxZggO9U4CC0xhS9f2or4wdxgODPs1M8eGuSfZM7tnjL8Bt-LvwU7-eYeXbWmzb9meaJkiPBfitStpNkzQynH69KlODRFDBs_y8ZkIontAmlB0RTW98kaiEcGrt3GO1iMmyaGNCIC6gbTfKKxJYD3pbcPQktZ5e2vXbAo4MOJCqsa8WMMRnPjxgqtupuGrfvaF3y2igtVI4zQ9ZHcVU95U8g4VrrdbWBL_HOtoqXGm70Ak67Xa48Au9jy-0VzioLvURbcSOZQp9VVfsX5t60pfYVRdyaUAhPfs1-IVGILjAb1adq6Is5nIJ6_tQUd1NgcYx1dGejXUCW6ULK5z4WDNXu-nAS4818zfO1kYwc9wx6iDUIMgXPEOda2rdJr52IC0CscwlWTFep2nOWtKgDoms-phpjAhqWnPznwiGDmoxVjcCEYmik2mVfo0a5NVX1qj36z3d3s6p_U9bXp5mmtMO9gIy2EwyWDjuXxna7qOzpwBuEnGDdpRwxaeYyl0eZB0qI7nMHQsN9vy7h1xVx-TVl9cfeBtOfdASuQLKAQ-8xN3jAjt64Zv12NTiCsjNeIIuw29IEOvbtCvffi3rPsQeAcVUlalkrOGPlvNmZIiWrn-2q8qypHPo89VHoYVbxtw251nUVJK0adguOffALoc='

In [16]:
decripted_msg = decrypt_asym(encrypted_msg, private_key)
decripted_msg

'This message was encripted with asymmetric cryptography'

# Simple Hash Functions

https://en.wikipedia.org/wiki/Hash_function

In [20]:
msg = "This is a long contract fdsf s fsdf "

In [21]:
def super_simple_hash(msg):
    """Just the first letter of the message"""
    return msg[0]

In [22]:
super_simple_hash(msg)

'T'

In [23]:
msg = "This is a totaly different contract :-)"

In [24]:
super_simple_hash(msg)

'T'

In [25]:
def hash_msg(msg=""):
    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [28]:
hash_msg("This is a long contract.")

'2ea211886896ac71f4d0d007c3f26908973a7681b8d51d986c14c8ab06c85966'

In [29]:
hash_msg("This is a totaly different contract :-)")

'fcaa045d6064a9f33c7a8677664f53ff60b0ce575979219d414a60e3129afe09'

In [30]:
def check_msg(msg, signed_hash):
    calculated_hash = hash_msg(msg)
    if signed_hash == calculated_hash:
        return True
    else:
        return False

In [26]:
msg = "This is my Contract"
msg_hash = hash_msg(msg)
check_msg(msg, msg_hash)

True

In [27]:
msg = "I change the contract so it is much better now for me :-)"
check_msg(msg, msg_hash)

False

# Digital signing messages

https://en.wikipedia.org/wiki/Digital_signature

In [31]:
def sign_msg(message, private_key):
    signature = private_key.sign(message, padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH), hashes.SHA256())
    return signature

In [32]:
msg = b"A message I want to sign"
signature = sign_msg(msg, private_key)

In [33]:
signature

b"}N\xab(\xf9F\x8c1\x1b%>\x12h$\xce=*\xbag\xf9\xf3i\x88\xd6\x07\x96\xe1?\xfc\xd9\xed\x8fR\xd1\xe3\x8dg\xe5A\x1c_5\xc9\xf4\x98\r5H$*\xbe/z<)\x1e\x93\x03'\x89F:\xa0c#\x1aP\xd2\xee\xd19\xf7\x13\x04\xa2\x8d\xf2\rm\x85\x11\x9a\xe6\xfe\xbe\xcaB\x8e<\xdd\xbd\x9e\xe7\xf725H\xa8P\xcdeu$s\xb9\x0e\xd2\xaf\x03\x99\x94\xe8\xf0\x97\x95U\x81v\xaeof8\x07?\xa0&\x02\xaa\xceK\xf4r\x17\x9d\t\xedh\x97\nN\xf8\x9b\xbe~&c.\xf99\x88\xe8\x9b\xb5\x80z\xc9\xaf\x1f5l\xf5\xf0\xc7%|\x92F\xf6\x9b7\x87)9O\x8b\xed\xf2\x15//$c\xd9|\xdb%\xc0\xbfN\xcf\x13\x00\xcc\xbf22>\x1b\xb3\x96\xe4\xd2>WJ\xb5h\x08@\x14<\x0e\xb8\xeflf}\xf3\xc2\x1d\xd7\x10\x93Q:0(oaH\xab\x03\xd7\xe6rxf\xab)\xed\xd1\x17R\xb9A\xbe\xa9sS\xe9\x0f\x00\xbf\x8d\xe5\x80\x1d\xad\xcd\x98\x95GgK\x19\x18\xfc\x98\xa2\x1e(\x81\xb9\x87{\xe9_\x8b\xea\x02\xa6\xb0\xc8\x1f\x1a\xb42\xd8\x13]\x91\xa1\x1f\xfa\xe0}\x7f\x98\xc8\x0f\xf9,\xf67\x8a\xb0$\xa36K\x07\x0f\xb0\x7fpw\x0b\x9f2\xa0\x11\xfe~G\x9aVi\xf3\xb2\xc4v'\x80-\xcb\xac:\x81\xdd\xc9\xee\xfa\x0fE\x11d\xca\x96%'\x9a\x98

In [34]:
def verify(signature, message, public_key):
    try:
        public_key.verify(signature, message, padding.PSS( mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH), hashes.SHA256())
        return True
    except: 
        return False

In [35]:
verify(signature, msg, public_key)

True

In [36]:
another_msg = "Another Message"

In [37]:
verify(signature, another_msg, public_key)

False

In [38]:
bob_private_key, bob_public_key = generate_keys()

In [39]:
verify(signature, msg, bob_public_key)

False